In [10]:
import os
import requests
KEY = '91mTqXFEeSaCIbjC0fuaB1RABk4HANqS4I91qRBN'

In [9]:
os.system('curl "https://api.propublica.org/congress/v1/115/senate/members.json" -H "X-API-Key:' + KEY + '"')

0

In [17]:
def getJSON(webpage):
    headers = {
        'X-API-Key': KEY
    }
    r = requests.get(webpage, headers=headers)
    return r.json()

getJSON('https://api.propublica.org/congress/v1/115/senate/members.json')


{'copyright': ' Copyright (c) 2017 Pro Publica Inc. All Rights Reserved.',
 'results': [{'chamber': 'Senate',
   'congress': '115',
   'members': [{'api_uri': 'https://api.propublica.org/congress/v1/members/A000360.json',
     'crp_id': '',
     'cspan_id': '5',
     'domain': '',
     'dw_nominate': '',
     'facebook_account': 'senatorlamaralexander',
     'first_name': 'Lamar',
     'google_entity_id': '/m/01rbs3',
     'govtrack_id': '300002',
     'icpsr_id': '40304',
     'id': 'A000360',
     'ideal_point': '',
     'in_office': 'true',
     'last_name': 'Alexander',
     'leadership_role': '',
     'lis_id': 'S289',
     'middle_name': '',
     'missed_votes': '1',
     'missed_votes_pct': '1.18',
     'next_election': '2020',
     'ocd_id': 'ocd-division/country:us/state:tn',
     'office': '455 Dirksen Senate Office Building',
     'party': 'R',
     'phone': '202-224-4944',
     'rss_url': 'http://www.alexander.senate.gov/public/?a=RSS.Feed',
     'senate_class': '2',
     '